In [ ]:
from IPython.core.display import HTML
import jieba


class Searcher_iindex():
    def __init__(self, title_file):
        self.index = dict()    
        self.max_id = 0
        self.title_list = [] 
        with open(title_file, 'r') as f:
            titles = f.read()
        for title in titles.split():
            self.add_doc(title)
    
    def add_doc(self, doc):
        self.title_list.append(doc)
        for term in list(jieba.cut(doc)):
            if term in self.index: 
                self.index[term].add(self.max_id)
            else:
                self.index[term] = set([self.max_id])
        self.max_id += 1
        return self.max_id - 1

    def word_match(self, word):
        result = None
        for term in list(jieba.cut(word)):
            if result is None:
                result = self.index.get(term, set())
            else:
                result = result & self.index[term]
        if result is None:
            result = set()
        return result

    def conv_query(self, query):
        query_new_parts = []
        all_parts = list(jieba.cut(query))
        idx = 0
        cache = ''
        while idx < len(all_parts):
            if all_parts[idx] == '(' or all_parts[idx] == ')':
                query_new_parts.append(all_parts[idx])
            elif all_parts[idx] == ' ':
                query_new_parts.append(' ')
            elif all_parts[idx] in ('and', 'AND'):
                query_new_parts.append('&')
            elif all_parts[idx] in ('or', 'OR'):
                
                query_new_parts.append('|')
            elif all_parts[idx] in ('not', 'NOT'):
                query_new_parts.append('-')
            elif all_parts[idx+1] not in (' ', ')'):
                cache += all_parts[idx]
            else:
                query_new_parts.append("self.word_match('{}')".format(cache + all_parts[idx]))
                cache = ''
            idx += 1
            
        for i in range(1,len(query_new_parts)-1):
            if query_new_parts[i]==' ':
                if not (query_new_parts[i-1] in ['(',')','&','|','-'] or 
                        query_new_parts[i+1] in ['(',')','&','|','-']):
                    query_new_parts[i]='&'
                    
        query_new = ''.join(query_new_parts)
        return query_new

    def highlighter(self, doc, word):  
        for part in list(jieba.cut(word)):
            if part not in ('(', ')', 'and', 'AND', 'or', 'OR', 'NOT', 'not', ' '):
                doc = doc.replace(part, '<span style="color:red">{}</span>'.format(part))
        return doc

    def search(self, query):
        result = []
        query_new = self.conv_query(query)
        for did in eval(query_new):
            result.append(self.highlighter(self.title_list[did], query))
        return result
    
    

In [72]:
searcher=Searcher_iindex('tiles.txt')

In [75]:
res=searcher.search('(苹果 芯片)')

In [76]:
for i in res:
    display(HTML(i))